<a href="https://colab.research.google.com/github/misabhishek/gcp-iam-recommender/blob/main/iam_recommender_Create_custom_role_based_on_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Statement

Create a custom role based on the permissions' usage of given `principals` in my entire organization

### Imports

In [ ]:
from google.colab import auth
import json
import subprocess

import concurrent
import logging

logging.basicConfig(format="%(levelname)s[%(asctime)s]:%(message)s")

### Lets authenticate

In [ ]:
auth.authenticate_user()
print('Authenticated')

### Helper functions

In [ ]:
def execute_command(command):
  return json.loads(subprocess.check_output(filter(lambda x: x, command.split(" "))).decode("utf-8"))

### Enter your org details

In [ ]:
organization = "# YOUR-ORGANIZATION-ID"
billing_project_id = "GCP-PROJECT-FOR-CALLING-THE-APIs

### Get All the projects

In [ ]:
get_all_projects_command = f"""gcloud asset search-all-resources \
                               --asset-types=cloudresourcemanager.googleapis.com/Project \
                               --scope={organization} --format=json --project={billing_project_id}"""

In [ ]:
def get_all_projects():
  projects = execute_command(get_all_projects_command)
  return [p["additionalAttributes"]["projectId"] for p in projects]

In [ ]:
project_ids = get_all_projects()

In [ ]:
print("project-ids\n\n", "\n".join(project_ids[:10]))

### Get All Insights

In [ ]:
insight_command = """gcloud recommender insights list \
    --project={} \
    --location=global \
    --insight-type=google.iam.policy.Insight \
    --format=json \
    --billing-project=""" + billing_project_id

In [ ]:
def get_all_insights():
  def get_insights(project_id):
    try:
      return {"project_id":project_id,
              "insights": execute_command(insight_command.format(project_id))}
    except:
      logging.warning(f"You don't have permissions to access project:`{project_id}`")
      return {"project_id": project_id,
              "insights": []}

  with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    insights = {"all_insights": list(executor.map(get_insights, project_ids))}
  
  return insights

In [ ]:
insights = get_all_insights()

### Get All Needed Permisisons

In [ ]:
def get_all_needed_permission_by_principals(principals):
  get_member = lambda insight:insight["content"]["member"] 

  def get_needed_permissions(insight):
    permissions = (insight["content"]["exercisedPermissions"] 
                   + insight["content"]["inferredPermissions"])
    return [p["permission"] for p in permissions]

  needed_permissions = []
  for project_insight in insights["all_insights"]:
    for insight in project_insight["insights"]:
      if get_member(insight) in principals:
        needed_permissions.extend(get_needed_permissions(insight))
  return needed_permissions

In [ ]:
all_needed_permissions = get_all_needed_permission_by_principals(
    {"user:misabhishek@google.com",
     "user:admin@iam-condition-demo.joonix.net"})

In [ ]:
all_needed_permissions[:10]

### Create a custom role using the needed permissions 

In [ ]:
#@title
